# Bachelor 2021

In [142]:
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.stats import norm
import scipy

In [594]:
S0 = 100
K = 100
T = 1
r = 0.07
sigma = 0.2
n_simulations = 1000000
n_steps = 252

In [ ]:
dt = T/n_steps
mu = (r-(sigma**2)/2)*dt
a = sigma*np.sqrt(dt)
x = np.random.normal(0,1,(n_simulations, n_steps))
# Her skal evt. udvides med flere/andre processer ift. Geometrisk Brownian Motions
# Formel kommer fra Paul Glasserman, 2004, s. 94

In [595]:
def geo_mean(iterable):
    temp = np.array(iterable)
    return temp.prod()**(1.0/len(temp))

def geo_mean_overflow(iterable):
    temp = np.log(iterable)
    return np.exp(temp.mean())

In [596]:
# Udvikling i prisen på det underliggende aktiv
smat = np.zeros((n_simulations, n_steps))
smat[:,0] += S0
for i in range(1, n_steps):
    smat[:,i] += smat[:,i-1]*np.exp(mu+a*x[:,i])

# Gennemsnit af prisen på det underliggende aktiv per simulering
avg = []
for i in range(len(smat)):
    avg.append(geo_mean_overflow(smat[i,:]))
    
# Beregning af payoff for call
c = []
for i in range(len(avg)):
    c.append(avg[i] - K)
    if c[i] < 0:
        c[i] = 0
    else: 
        c[i] = c[i]

# Beregning af payoff for put
p = []
for i in range(len(avg)):
    p.append(K - avg[i])
    if p[i] < 0:
        p[i] = 0
    else: 
        p[i] = p[i]

# Gennemsnitligt payoff
payoff_call = np.mean(c)
payoff_put = np.mean(p)

# Tilbagediskontering
call = payoff_call*np.exp(-r*T)
put = payoff_put*np.exp(-r*T)

In [605]:
call

6.0015919581525825

In [606]:
put

3.018142411329767

**Tjek med Black-Scholes**

$C_G = S_0 e^{(b-r)T} \Phi(d_1) - K e^{-rT}\Phi(d_2)$

$P_G = K e^{-rT}\Phi(-d_2) - S_0 e^{(b-r)T} \Phi(-d_1)$

$\sigma_G = \frac{\sigma}{\sqrt{3}}$

$b = \frac{1}{2}(r-\frac{1}{2} \sigma_G^2)$

$d_1 = \frac{log(\frac{S_0}{K})+(b+\frac{1}{2}\sigma_G^2)T}{\sigma_G\sqrt{T}}$

$d_2 = d_1 - \sigma_G \sqrt{T}$

In [602]:
sigma_G = sigma/np.sqrt(3)

b = (1/2)*(r-(1/2)*(sigma_G**2))

d1 = (np.log(S0/K)+(b+1/2*(sigma_G**2))*T)/(sigma_G*np.sqrt(T))

d2 = d1 - sigma_G*np.sqrt(T)

C_G = S0*np.exp((b-r)*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

P_G = K*np.exp(-r*T)*norm.cdf(-d2) - S0*np.exp((b-r)*T)*norm.cdf(-d1)

In [603]:
C_G

6.024544283554853

In [604]:
P_G

3.024717268581746